In [1]:
import pandas as pd
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [2]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
model_name='multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-28 11:12:46.306536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 11:12:47.128449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gary/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the avail

In [5]:
answer_llm = df.iloc[0].answer_llm

In [6]:
embs = embedding_model.encode(answer_llm)

In [7]:
for i in embs:
    print("{:f}".format(i))
    break

-0.422447


In [8]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm =  record['answer_llm']

    v_orig = embedding_model.encode(answer_orig)
    v_llm = embedding_model.encode(answer_llm)

    return v_llm.dot(v_orig)

In [9]:
results_gpt4o = df.to_dict(orient='records')

In [10]:
from tqdm import tqdm
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity(record)
    similarity.append(sim)

100%|██████████| 300/300 [00:44<00:00,  6.67it/s]


In [11]:
df['cosine'] = similarity

In [12]:
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine, dtype: float64

In [21]:
import numpy as np

In [24]:
norm = np.sqrt(( df['cosine'] *  df['cosine']).sum())

In [25]:
norm

488.87384

In [26]:
v_norm = df['cosine'] / norm

In [28]:
v_norm.describe()

count    300.000000
mean       0.056244
std        0.013060
min        0.009303
25%        0.049722
50%        0.057964
75%        0.064790
max        0.080749
Name: cosine, dtype: float64

In [13]:
df.head()

,answer_llm,answer_orig,document,question,course,cosine
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,17.515987
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,13.418402
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,25.313255
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,12.147415
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,18.747736
